In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy as dc
from torch.utils.data import DataLoader
import glob
import random


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

file = 'archive/YUM.csv'



In [8]:

files = glob.glob("archive/*.csv")
# random.shuffle(files)
# files

<h2>Model</h2>

<h2>Training</h2>

In [9]:
from Modules.train import train_model

In [10]:

# Load data into pytorch dataset

from Modules.dataset_class import TimeSeriesDataset
from Modules.preprocess import process_data
def train_on_file(file, model, num_epochs, loss_function, optimizer, device):

    print("PROCESSING "+ file)
    _, _, X_train, X_test, y_train, y_test, _ = process_data(file)
    train_dataset = TimeSeriesDataset(X_train, y_train)
    test_dataset = TimeSeriesDataset(X_test, y_test)
    X_train.shape, X_test.shape, y_train.shape, y_test.shape

    # create batches
    batch_size = 16

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = train_model(num_epochs, model, loss_function, optimizer, train_loader,test_loader, device)
    return model

In [11]:
def train_all(files, input_model, num_epochs, loss_function, optimizer, device):
    model = input_model
    save = 1
    for file in files:
        model = train_on_file(file, model, num_epochs, loss_function, optimizer, device)
        if save % 5 == 0:
            torch.save(model, 'checkpoints/forecast'+str(save)+'.pt')
        save += 1

    torch.save(model, 'forecast_final.pt')

In [12]:

from Modules.model import LSTM

model = LSTM(30, 1, 64, 16, 1)
learning_rate = 0.001
num_epochs = 10
loss_function = nn.HuberLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# files = sorted(glob.glob("archive/*.csv"))[:10]
files = glob.glob("archive/*.csv")
random.shuffle(files)

files_list_path = "checkpoints/filenames.txt"
with open(files_list_path, 'w') as file:
    # Write each element of the string array to the file
    for item in files:
        file.write(item + '\n')



train_all(files, model, num_epochs, loss_function, optimizer, device)

PROCESSING archive/CAH.csv
Epoch: 1
Batch 100, Loss: 0.0415073
Batch 200, Loss: 0.0103517
Batch 300, Loss: 0.0003691
Batch 400, Loss: 0.0001844
Batch 500, Loss: 0.0001847

Val Loss: 0.0004351
***************************************************

Epoch: 2
Batch 100, Loss: 0.0001252
Batch 200, Loss: 0.0001862
Batch 300, Loss: 0.0001437
Batch 400, Loss: 0.0001851
Batch 500, Loss: 0.0001402

Val Loss: 0.0004768
***************************************************

Epoch: 3
Batch 100, Loss: 0.0001134
Batch 200, Loss: 0.0001267
Batch 300, Loss: 0.0001398
Batch 400, Loss: 0.0001189
Batch 500, Loss: 0.0001604

Val Loss: 0.0002472
***************************************************

Epoch: 4
Batch 100, Loss: 0.0000900
Batch 200, Loss: 0.0000898
Batch 300, Loss: 0.0001078
Batch 400, Loss: 0.0000856
Batch 500, Loss: 0.0001368

Val Loss: 0.0002792
***************************************************

Epoch: 5
Batch 100, Loss: 0.0000893
Batch 200, Loss: 0.0001054
Batch 300, Loss: 0.0001007
Batch 400, 